# Trinkgeld Calculator

In [166]:
import pandas as pd
from datetime import datetime

In [167]:
class WorkDay:
    def __init__(self, day, hours):
        self.day = datetime.strptime(day[3:], "%d.%m.%Y")
        self.hours = self._parse_hours(hours)
    
    def add(self, workday):
        if (self.day != workday.day):
            raise ValueError('Days of workdays must match to sum up work hours!')
        else:
            self.hours += workday.hours
    
    def _parse_hours(self, hours):
        hours, minutes = map(int, hours.split(':'))
        return round(hours + (minutes / 60), 2)      
    
    def __repr__(self):
        return f"{self.day}: {self.hours}"
    
class Employee:
    def __init__(self, name, work_days):
        self.name = name
        self.work_days = work_days

    def __repr__(self):
        work_days = '\n'.join('\t' + str(work_day) for work_day in self.work_days)
        return f"{self.name}: \n{work_days}"

def parse_hours_for(name):
    sheet = xl.parse(name, header=None)
    header = sheet[sheet[0] == 'Tag'].index[0]

    data = xl.parse(name, header=header)
    data = data[['Tag', 'Startzeit', 'Endzeit', 'Pause (min)', 'Dauer netto (h)']]
    data = data[data['Startzeit'].notnull()]
    
    work_days = {}
    for _, row in data.iterrows():
        work_day = WorkDay(day=row['Tag'], hours=row['Dauer netto (h)'])
        if work_day.day in work_days:
            work_days[work_day.day].add(work_day)
        else:
            work_days[work_day.day] = work_day

    employee = Employee(name=name, work_days=work_days.values())

    return employee


In [168]:
class DailyWorkHours:
    def __init__(self, tip):
        self.tip = tip
        self.work_hours = {}

    def total_hours(self):
        return round(sum(self.work_hours.values()), 2)

    def __repr__(self):
        total_hours = self.total_hours()
        tip_per_hour = round(self.tip / total_hours, 2)
        work_hours = '\n'.join([f'\t{name}:\n\t\thours {hours}\n\t\ttip {round(hours * tip_per_hour, 2)}' for name, hours in self.work_hours.items()])
        return f'total tip {self.tip}, total hours {total_hours}, tip per hour {tip_per_hour}\n{work_hours}'
    
class WeeklyTips:
    def __init__(self, daily_tips, employees_work_hours):
        days = sorted({work_day.day for employee in employees_work_hours for work_day in employee.work_days})
        self.weekly_work_times = {day: DailyWorkHours(tip) for day, tip in zip(days, daily_tips)}
        for employee in employees_work_hours:
            for work_day in employee.work_days:
                daily_work_hours = self.weekly_work_times[work_day.day]
                daily_work_hours.work_hours[employee.name] = work_day.hours

    def __repr__(self):
        return '\n'.join([f'{day.strftime("%d.%m.%Y")}: {daily_work_times}' for day, daily_work_times in self.weekly_work_times.items()])

In [169]:
# Change these variables
path = ('./data/sample01.xlsx') # path to weekly work hours
daily_tips = [112.00, 177.71, 155.20, 96.22, 143.50, 181.50, 160.90] # tips MO-SO

xl = pd.ExcelFile(path)
employees = xl.sheet_names
employees.remove('Übersicht')
employees_work_hours = [parse_hours_for(name) for name in employees]

weekly_work_times = WeeklyTips(daily_tips=daily_tips, employees_work_hours=employees_work_hours)
print(weekly_work_times)

10.03.2025: total tip 112.0, total hours 35.43, tip per hour 3.16
	Açelya Basili:
		hours 4.52
		tip 14.28
	Anna Sophie Blessing:
		hours 6.0
		tip 18.96
	Laura Greidenweiß:
		hours 5.78
		tip 18.26
	Mauro Sirigu:
		hours 11.7
		tip 36.97
	Jennifer Wöhrlin:
		hours 7.43
		tip 23.48
11.03.2025: total tip 177.71, total hours 60.27, tip per hour 2.95
	Emelie Appl:
		hours 5.05
		tip 14.9
	Manuel Bär:
		hours 7.98
		tip 23.54
	Markus Fuchs:
		hours 6.12
		tip 18.05
	Ekaterina Grashchenko:
		hours 7.25
		tip 21.39
	Philine Ostermayer:
		hours 8.38
		tip 24.72
	Damiano Parziale:
		hours 7.7
		tip 22.72
	Mauro Sirigu:
		hours 11.92
		tip 35.16
	Matthias Strobel:
		hours 5.87
		tip 17.32
12.03.2025: total tip 155.2, total hours 52.9, tip per hour 2.93
	Emelie Appl:
		hours 5.93
		tip 17.37
	Fabrizio Bellia:
		hours 11.0
		tip 32.23
	Anna Sophie Blessing:
		hours 5.82
		tip 17.05
	Sabulla Bullach:
		hours 7.13
		tip 20.89
	Markus Fuchs:
		hours 7.02
		tip 20.57
	Marie Jaussi:
		hours 8.32
		tip